# <center>Visual Analysis of Gun tweets and Gun stocks</center>
<center> Lisa Phung  <br> <center>


***


In [ ]:
#imports
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import json
import csv
from textblob import TextBlob
import seaborn as sns
%matplotlib inline
from datetime import datetime
import numpy as np
from pandas import Timestamp


#### Tweepy API grabbed tweets using certain keywords such as guns, 2nd Amendment, and firearms for a given amount of time. Each tweet and the time it was tweeted is written to a new csv file.

In [ ]:


ckey = "kLEVLIO4HHblzsmhooY7y0ezV"
csecret = "MM3PAEN9Rjxu0Kgnil9eA37ET4cCcfaoTvMmmkShLYDLz3FOKn"
atoken = "235327323-aPVX5Tl022C8a2pfSllVNHrsihKL4D9kXYUYyeWI"
asecret = "dPaSPwBil04uhjtKynCkKaqnCgcTaucL74q3zAeAexVC0"

textList = []
dateList = []
class MyStreamListener(StreamListener):
    def __init__(self, time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.saveFile = open('abcd.json', 'a')
        super(MyStreamListener, self).__init__()
        self.filename = 'data' + '_' + time.strftime('%Y%m%d-%H%M%S') + '.csv'
        # Create a new file with that filename
        csvFile = open(self.filename, 'w', encoding='utf-8')

        # Create a csv writer
        csvWriter = csv.writer(csvFile)

        #Write a single row with the headers of the columns
        csvWriter.writerow(['text','created_at'])


    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            all_data = json.loads(data)
            tweet = all_data["text"]
            username = all_data["user"]["screen_name"]
            date = all_data["created_at"]
            # Open the csv file created previously
            csvFile = open(self.filename, 'a', encoding='utf-8')

            # Create a csv writer
            csvWriter = csv.writer(csvFile)

            csvWriter.writerow([tweet, date])

            # Close the csv file
            csvFile.close()


            #print((username, tweet))
            return True
        else:
            print("Time is up!")
            return False

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

myStream = Stream(auth, listener=MyStreamListener(time_limit=10)) #Stream will stay open for 10 seconds (or whatever integer you pass)
myStream.filter(track=['guns', '2ndAmendment', 'firearms'])

#### Below is the csv which was created using the tweepy api to stream tweets for certain keywords

In [ ]:
df = pd.read_csv("data_20171211-153014.csv",encoding = "ISO-8859-1", names=['Tweet', 'Created At'])
df

#### Combine Stream Tweet Data

In [ ]:

filenames = ["data_20171211-153007.csv",
             "data_20171212-153008.csv", "data_20171213-153003.csv", 
             "data_20171212-153206.csv", "data_20171211-153014.csv"] # Fill in remaining files.


#combines all the data into one csv
header_saved = False
with open('output.csv','w', encoding='ISO-8859-1') as fout:
    for filename in filenames:
        with open(filename, encoding='ISO-8859-1') as fin:
            header = next(fin)
            if not header_saved:
                fout.write(header)
                header_saved = True
            for line in fin:
                fout.write(line)

#Combines data for just Monday
filenames = ["data_20171211-153007.csv","data_20171211-153014.csv"] # Fill in remaining files.

header_saved = False
with open('monday.csv','w', encoding='ISO-8859-1') as fout:
    for filename in filenames:
        with open(filename, encoding='ISO-8859-1') as fin:
            header = next(fin)
            if not header_saved:
                fout.write(header)
                header_saved = True
            for line in fin:
                fout.write(line)

#Combines data for just Tuesday
filenames1 = ["data_20171212-153008.csv","data_20171212-153206.csv",] # Fill in remaining files.

header_saved = False
with open('tuesday.csv','w', encoding='ISO-8859-1') as fout:
    for filename in filenames1:
        with open(filename, encoding='ISO-8859-1') as fin:
            header = next(fin)
            if not header_saved:
                fout.write(header)
                header_saved = True
            for line in fin:
                fout.write(line)
                
#Combines data for just Wednesday                
filenames2 = ["data_20171213-153003.csv"] # Fill in remaining files.

header_saved = False
with open('wednesday.csv','w', encoding='ISO-8859-1') as fout:
    for filename in filenames2:
        with open(filename, encoding='ISO-8859-1') as fin:
            header = next(fin)
            if not header_saved:
                fout.write(header)
                header_saved = True
            for line in fin:
                fout.write(line)

#### Sentiment analysis using the tweets from the csv file. 

In [ ]:

#create dataframe object from csv file
df = pd.read_csv('output.csv')
#df1 = pd.read_csv('data_20171212-153311.csv')

#create variable name for column that contains text of tweets
text = df['text']
sentimentlist = []



#iterate through each tweet in the first column of csv
for row in df['text']:
    analysis = TextBlob(row)
    
    #add sentiment scores to a list
    sentimentlist.append(analysis.sentiment.polarity)

    #add a column in dataframe with sentiment scores
df["Sentiment Score"] = sentimentlist

#make a new column that changes the time to integers
df['dateInt'] = pd.to_datetime(df.created_at)
df.sort_values(by='dateInt')

#calculates moving average of Sentiment score
df['Moving Average'] = df['Sentiment Score'].rolling(5).mean()

#creates line graph over time of Sentiment Score
df.plot('dateInt', 'Moving Average')
# df.plot('dateInt', 'Sentiment Score')





#### Read in tweet and stock data. 

In [ ]:

#create dataframe object from csv file
df = pd.read_csv('monday.csv')
df1 = pd.read_csv('tuesday.csv')
df2 = pd.read_csv('wednesday.csv')
#df4 = pd.read_csv('output.csv')
df3 = pd.read_csv("RGRdata.csv",encoding = "ISO-8859-1", names=['someNum', 'close', 'high','low','open','volume','date'])
df3.drop(df.head(8).index,inplace=True)
df3 = df3[:-1]
# print(MonHighRGR = df3.iloc[229:257, 'high'])
# print(MonDateRGR = df3.iloc[229.257, 'date'])

RGR = df3[['high', 'date']].copy()

MonRGR = RGR.iloc[229:257]
TuesRGR = RGR.iloc[476:505]
WedRGR = RGR.iloc[766:794]


#pd.options.mode.chained_assignment = None  # default='warn'

#create variable name for column that contains text of tweets
# text = df['text']
# text1 = df1['text']
# text2 = df2['text']
#text4 = df4['text']
sentimentlist = []
sentimentlist1 = []
sentimentlist2 = []
sentimentlist4 = []

#create

#iterate through each tweet in the first column of csv
for row in df['text']:
    analysis = TextBlob(row)
    
    #add sentiment scores to a list
    sentimentlist.append(analysis.sentiment.polarity)
for row in df1['text']:
    analysis = TextBlob(row)
    
    #add sentiment scores to a list
    sentimentlist1.append(analysis.sentiment.polarity)
for row in df2['text']:
    analysis = TextBlob(row)
    
     #add sentiment scores to a list
    sentimentlist2.append(analysis.sentiment.polarity)
    
# for row in df4['text']:
#     analysis = TextBlob(row)
    
#     #add sentiment scores to a list
#     sentimentlist4.append(analysis.sentiment.polarity)


    #add a column in dataframe with sentiment scores
df["Sentiment Score"] = sentimentlist
df1["Sentiment Score"] = sentimentlist1
df2["Sentiment Score"] = sentimentlist2
#df4["Sentiment Score"] = sentimentlist4

# list_temp = []
# for row in df4['created_at']:
#     list_temp.append(Timestamp(row, tz = 'UTC').tz_convert('US/Eastern'))
# df4['created_at'] = list_temp

list_temp1 = []
for row in df3['date']:
    list_temp1.append(Timestamp(row, tz = 'EST').tz_convert('US/Eastern'))
df3['date'] = list_temp1

#make a new column that changes the time to integers
df['dateInt'] = pd.to_datetime(df.created_at)
df1['dateInt'] = pd.to_datetime(df1.created_at)
df2['dateInt'] = pd.to_datetime(df2.created_at)
#df4['date'] = pd.to_datetime(df4.created_at)
df3['date'] = pd.to_datetime(df3.date)
MonRGR['date'] = pd.to_datetime(MonRGR.date)
TuesRGR['date'] = pd.to_datetime(TuesRGR.date)





#calculates moving average of Sentiment score
df['Moving Average'] = df['Sentiment Score'].rolling(5).mean()
df1['Moving Average'] = df1['Sentiment Score'].rolling(5).mean()
df2['Moving Average'] = df2['Sentiment Score'].rolling(5).mean()
#df4['Moving Average'] = df4['Sentiment Score'].rolling(5).mean()
#creates line graph over time of Sentiment Score
#df.plot('dateInt', ['Moving Average', '')
#RGR.plot('dateInt', 'high')
#df2.plot('dateInt', 'Moving Average')
# df.plot('dateInt', 'Sentiment Score')

#merge = df.append(MonRGR, ignore_index=True)
#merge.to_csv('file_name.csv')



#merged.to_csv('file_name.csv')

# MonRGR.high = MonRGR.high.astype(float)
# df3.high = df3.high.astype(float)
# df3[229:256].set_index('date')['high'].plot()
# df.plot('dateInt', 'Moving Average')

#### Plot Monday Tweets and Stock

In [ ]:
fig = plt.figure(1)
fig.set_size_inches(12, 4)

plt.subplot(121)
plt.title('Monday Stock of Sturm, Ruger & Company, Inc')
plt.ylabel('Stock')
plt.xlabel('Date and Time')
plt.plot(df3[229:256].set_index('date')['high'])

plt.subplot(122)
plt.title('Sentiment Scores of Tweets from 12/11/17')
plt.ylabel('Sentiment Score')
plt.xlabel('Date and Time')
plt.plot(df['dateInt'], df["Moving Average"])
plt.show()


#### Plot Tuesday Tweets and Stock

In [ ]:
fig = plt.figure(1)
fig.set_size_inches(12, 4)

plt.subplot(121)
plt.title('Tuesday Stock of Sturm, Ruger & Company, Inc')
plt.ylabel('Stock')
plt.xlabel('Date and Time')
plt.plot(df3[476:505].set_index('date')['high'])

plt.subplot(122)
plt.title('Sentiment Scores of Tweets from 12/12/17')
plt.ylabel('Sentiment Score')
plt.xlabel('Date and Time')
plt.plot(df1['dateInt'], df1["Moving Average"])
plt.show()

#### Plot Wednesday Tweets and Stock

In [ ]:
fig = plt.figure(1)
fig.set_size_inches(12, 4)

plt.subplot(121)
plt.title('Wednesday Stock of Sturm, Ruger & Company, Inc')
plt.ylabel('Stock')
plt.xlabel('Date and Time')
plt.plot(df3[766:794].set_index('date')['high'])

plt.subplot(122)
plt.title('Sentiment Scores of Tweets from 12/13/17')
plt.ylabel('Sentiment Score')
plt.xlabel('Date and Time')
plt.plot(df2['dateInt'], df2["Moving Average"])
plt.show()